In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import PatternFill, Alignment, Font



In [2]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [3]:
# WebDriver'ı başlat
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # AutoScout24 URL'sine git
    driver.get("https://www.autoscout24.com.tr/lst/bmw/i7")

    # İsim, fiyat, km ve konum öğelerinin yüklenmesini bekle
    WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "ListItem_title__ndA4s"))
    )

    # İsim öğelerini bul
    car_names = driver.find_elements(By.CLASS_NAME, "ListItem_title__ndA4s")

    # Fiyat öğelerini bul
    car_prices = driver.find_elements(By.CLASS_NAME, "Price_price__APlgs")

    # Kilometre öğelerini data-testid ile bul
    car_kms = driver.find_elements(By.XPATH, "//span[@data-testid='VehicleDetails-mileage_road']")

    # Konum öğelerini bul
    car_locations = driver.find_elements(By.CLASS_NAME, "SellerInfo_address__leRMu")

    # İsim, fiyat, kilometre ve konum bilgilerini birleştirip liste oluştur
    car_data = []
    for name, price, km, location in zip(car_names, car_prices, car_kms, car_locations):
        # Eksik veri kontrolü
        car_name = name.text.strip() if name else "Unkown"
        car_price = price.text.strip() if price else "No Price"
        
        # Kilometre kontrolü: Eğer boş, `-` veya geçersizse "KM Yok" yaz
        car_km_raw = km.text.strip() if km else "-"
        car_km = car_km_raw if any(char.isdigit() for char in car_km_raw) else "KM Yok"
        
        car_location = location.text.strip() if location else "Konum Yok"

        # Verileri listeye ekle
        car_data.append({
            "Model": car_name,
            "Fiyat": car_price,
            "Kilometre": car_km,
            "Konum": car_location
        })

    # Verileri Pandas DataFrame'e dönüştür
    df = pd.DataFrame(car_data)

    # Excel'e kaydet
    excel_path = "C:\\Users\\90546\\Desktop\\Cari7.xlsx"
    df.to_excel(excel_path, index=False)
    print(f"Veriler Excel dosyasına başarıyla kaydedildi: {excel_path}")

finally:
    # Tarayıcıyı kapat
    driver.quit()

Veriler Excel dosyasına başarıyla kaydedildi: C:\Users\90546\Desktop\Cari7.xlsx


In [4]:
# Excel dosyasını yükle
file_path = "C:\\Users\\90546\\Desktop\\Cari7.xlsx"
workbook = load_workbook(file_path)

# İlk çalışma sayfasını seç
sheet = workbook.active

# Başlık renkleri ve font tanımları
header_colors = {
    "A": {"header": "4F81BD", "content": "D9E1F2"},  # Model: Koyu mavi başlık, açık mavi içerik
    "B": {"header": "C0504D", "content": "F4CCCC"},  # Fiyat: Kırmızı başlık, açık kırmızı içerik
    "C": {"header": "9BBB59", "content": "E2EFDA"},  # Kilometre: Yeşil başlık, açık yeşil içerik
    "D": {"header": "8064A2", "content": "D9D2E9"},  # Konum: Mor başlık, açık mor içerik
}
header_font = Font(bold=True, color="FFFFFF")  # Beyaz renkli ve kalın font
alignment = Alignment(horizontal="center", vertical="center")  # Ortalama

# Başlık hücrelerini düzenle ve renk uygula
for col_letter, colors in header_colors.items():
    # Başlık hücresini renklendir
    header_cell = sheet[f"{col_letter}1"]
    header_cell.fill = PatternFill(start_color=colors["header"], end_color=colors["header"], fill_type="solid")
    header_cell.font = header_font
    header_cell.alignment = alignment

    # İçerik hücrelerini renklendir
    for row in sheet.iter_rows(min_row=2, min_col=header_cell.column, max_col=header_cell.column):
        for cell in row:
            cell.fill = PatternFill(start_color=colors["content"], end_color=colors["content"], fill_type="solid")
            cell.alignment = alignment  # İçeriği ortala

            # "KM Yok" için özel bir renk uygula
            if col_letter == "C" and cell.value == "KM Yok":
                cell.fill = PatternFill(start_color="FF9999", end_color="FF9999", fill_type="solid")  # Pembe

# Sütun genişliğini ayarla
for column in sheet.columns:
    max_length = 0
    column_letter = get_column_letter(column[0].column)  # Sütun harfi (A, B, C...)
    for cell in column:
        try:
            # Hücredeki metnin uzunluğunu kontrol et
            if cell.value:
                max_length = max(max_length, len(str(cell.value)))
        except:
            pass
    # Sütun genişliğini ayarla
    sheet.column_dimensions[column_letter].width = max_length + 2

# Değişiklikleri kaydet
workbook.save(file_path)
print(f"Excel tablosu başarıyla düzenlendi: {file_path}")

Excel tablosu başarıyla düzenlendi: C:\Users\90546\Desktop\Cari7.xlsx


In [5]:
##df.to_excel("C:\\Users\\90546\\Desktop\\car.xlsx",index=False)